# Day15_0: 앙상블 & AutoML - 정답 노트북

---

In [ ]:
# 공통 라이브러리 임포트
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    VotingClassifier, StackingClassifier
)
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.datasets import load_iris, make_classification

import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료!")

---

## Q1. Hard Voting 이해하기 ⭐

**문제**: 3개 모델의 예측이 다음과 같을 때, Hard Voting 결과는?

In [ ]:
# 문제
model_a_pred = 1
model_b_pred = 0
model_c_pred = 1

In [ ]:
# 정답 코드
from collections import Counter

# 모든 예측을 리스트로
predictions = [model_a_pred, model_b_pred, model_c_pred]

# 다수결 투표
vote_counts = Counter(predictions)
hard_voting_result = vote_counts.most_common(1)[0][0]

print(f"각 모델 예측: {predictions}")
print(f"투표 결과: {dict(vote_counts)}")
print(f"Hard Voting 최종 예측: {hard_voting_result}")

In [ ]:
# 테스트
assert hard_voting_result == 1, "클래스 1이 다수결!"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
- Hard Voting은 각 모델의 예측 클래스를 모아 다수결로 결정
- Counter를 사용하여 각 클래스의 투표 수 계산

**핵심 개념**:
- Hard Voting: 가장 많이 예측된 클래스 선택
- 동점일 경우 일반적으로 작은 클래스 반환 (구현에 따라 다름)

**대안**:
```python
# numpy로 구현
import numpy as np
predictions = np.array([1, 0, 1])
result = np.bincount(predictions).argmax()
```

**실무 팁**: Hard Voting은 확률 정보를 활용하지 못하므로, Soft Voting을 더 권장

---

## Q2. Soft Voting 계산하기 ⭐⭐

**문제**: 3개 모델의 클래스 1 예측 확률로 Soft Voting 결과 계산

In [ ]:
# 문제
proba_a = 0.8  # 모델 A의 클래스 1 확률
proba_b = 0.3  # 모델 B의 클래스 1 확률
proba_c = 0.6  # 모델 C의 클래스 1 확률

In [ ]:
# 정답 코드
# 클래스 1 평균 확률
avg_proba_class1 = (proba_a + proba_b + proba_c) / 3

# 클래스 0 평균 확률
avg_proba_class0 = 1 - avg_proba_class1

# 최종 예측 (0.5 기준)
soft_voting_result = 1 if avg_proba_class1 >= 0.5 else 0

print(f"각 모델 클래스1 확률: {proba_a}, {proba_b}, {proba_c}")
print(f"평균 클래스0 확률: {avg_proba_class0:.4f}")
print(f"평균 클래스1 확률: {avg_proba_class1:.4f}")
print(f"Soft Voting 최종 예측: {soft_voting_result}")

In [ ]:
# 테스트
assert abs(avg_proba_class1 - 0.5667) < 0.01, "평균 확률 계산 확인"
assert soft_voting_result == 1, "클래스 1이 선택되어야 함"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
- 각 모델의 클래스별 확률을 평균
- 평균 확률이 높은 클래스를 선택

**핵심 개념**:
- Soft Voting은 확률을 평균하여 더 정교한 결합
- 불확실한 예측(확률 0.5 근처)의 영향을 줄일 수 있음

**대안**:
```python
# numpy 활용
probas = np.array([proba_a, proba_b, proba_c])
avg_proba = probas.mean()
```

**실무 팁**: 모델 성능에 따라 가중치를 다르게 부여하면 성능 향상 가능

---

## Q3. VotingClassifier 구성하기 ⭐⭐

**문제**: LogisticRegression과 RandomForest로 Soft Voting 앙상블 구성

In [ ]:
# 문제 - 데이터 준비
from sklearn.datasets import load_iris
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X, y = iris.data, (iris.target == 2).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 정답 코드
# 개별 모델 정의
lr = LogisticRegression(random_state=42, max_iter=1000)
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Soft Voting 앙상블 구성
voting_clf = VotingClassifier(
    estimators=[
        ('lr', lr),
        ('rf', rf)
    ],
    voting='soft'  # Soft Voting
)

# 학습 및 예측
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
print(f"Soft Voting 정확도: {accuracy:.4f}")

# 개별 모델 성능 비교
for name, model in [('LR', lr), ('RF', rf)]:
    model.fit(X_train, y_train)
    acc = accuracy_score(y_test, model.predict(X_test))
    print(f"{name} 정확도: {acc:.4f}")

In [ ]:
# 테스트
assert accuracy >= 0.9, "정확도 90% 이상"
assert hasattr(voting_clf, 'predict_proba'), "Soft Voting은 predict_proba 지원"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. 개별 모델을 튜플 리스트로 구성 `(이름, 모델)`
2. `voting='soft'`로 Soft Voting 지정
3. fit/predict 일반 모델처럼 사용

**핵심 개념**:
- estimators: (이름, 모델) 튜플 리스트
- voting: 'hard' 또는 'soft'
- Soft Voting 시 모든 모델이 predict_proba 지원해야 함

**흔한 실수**:
- SVM은 기본적으로 확률 미지원 -> `probability=True` 필요
- estimators에 이름 누락

**실무 팁**: 다양한 알고리즘 조합이 성능 향상에 효과적

---

## Q4. Stacking vs Voting 비교 ⭐⭐⭐

**문제**: 같은 베이스 모델로 Stacking과 Voting 성능 비교

In [ ]:
# 정답 코드
from sklearn.ensemble import StackingClassifier, GradientBoostingClassifier

# 베이스 모델 정의
base_estimators = [
    ('lr', LogisticRegression(random_state=42, max_iter=1000)),
    ('rf', RandomForestClassifier(n_estimators=50, random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=50, random_state=42))
]

# Voting Classifier
voting = VotingClassifier(
    estimators=base_estimators,
    voting='soft'
)

# Stacking Classifier
stacking = StackingClassifier(
    estimators=base_estimators,
    final_estimator=LogisticRegression(random_state=42),
    cv=5
)

# 학습 및 평가
results = {}

for name, model in [('Voting', voting), ('Stacking', stacking)]:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    print(f"{name} 정확도: {accuracy:.4f}")

print(f"\n성능 차이: {abs(results['Voting'] - results['Stacking']):.4f}")

In [ ]:
# 테스트
assert 'Voting' in results and 'Stacking' in results
assert all(v >= 0.8 for v in results.values()), "모든 모델 80% 이상"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. 동일한 베이스 모델로 Voting, Stacking 구성
2. Stacking은 final_estimator(메타 모델) 추가 필요
3. 동일 데이터로 성능 비교

**핵심 개념**:
- Voting: 단순 평균/다수결
- Stacking: 베이스 모델 예측을 특성으로 메타 모델 학습
- cv 파라미터: 메타 특성 생성 시 교차 검증 폴드 수

**대안**:
```python
# passthrough=True로 원본 특성 포함
stacking = StackingClassifier(..., passthrough=True)
```

**실무 팁**: Stacking이 항상 좋은 것은 아님. 과적합 위험 고려

---

## Q5. 가중 Voting 최적화 ⭐⭐⭐

**문제**: 개별 모델 성능에 비례하여 가중치 설정

In [ ]:
# 문제
lr_score = 0.85
rf_score = 0.90
gb_score = 0.92

In [ ]:
# 정답 코드
# 점수를 가중치로 변환 (정규화)
scores = [lr_score, rf_score, gb_score]
total_score = sum(scores)
weights = [s / total_score for s in scores]

# 또는 원래 점수를 그대로 가중치로 사용
weights_raw = [lr_score, rf_score, gb_score]

print(f"정규화 가중치: {[f'{w:.3f}' for w in weights]}")
print(f"원래 점수 가중치: {weights_raw}")

# Weighted Voting 구성
weighted_voting = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42, max_iter=1000)),
        ('rf', RandomForestClassifier(n_estimators=50, random_state=42)),
        ('gb', GradientBoostingClassifier(n_estimators=50, random_state=42))
    ],
    voting='soft',
    weights=weights_raw  # 성능 비례 가중치
)

weighted_voting.fit(X_train, y_train)
y_pred_weighted = weighted_voting.predict(X_test)
accuracy_weighted = accuracy_score(y_test, y_pred_weighted)

print(f"\nWeighted Voting 정확도: {accuracy_weighted:.4f}")

In [ ]:
# 테스트
assert len(weights) == 3, "3개 모델 가중치"
assert weights_raw[2] == max(weights_raw), "GB가 가장 높은 가중치"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. 각 모델의 CV 성능을 가중치로 사용
2. weights 파라미터에 리스트로 전달
3. 성능 좋은 모델의 영향력 증가

**핵심 개념**:
- weights: estimators 순서대로 가중치 리스트
- 정규화 여부는 선택 (sklearn은 자동 처리)
- 극단적 가중치는 피하는 것이 좋음

**실무 팁**: 가중치는 CV 점수 기반으로 설정하되, 최적 가중치는 실험으로 찾기

---

## Q6. Optuna 기본 사용 ⭐⭐⭐

**문제**: Optuna로 LogisticRegression의 C 파라미터 최적화

In [ ]:
# 정답 코드
import optuna
from sklearn.model_selection import cross_val_score

# 목적 함수 정의
def objective_lr(trial):
    """LogisticRegression C 파라미터 최적화"""
    
    # C 파라미터 탐색 (log uniform)
    C = trial.suggest_float('C', 0.01, 100, log=True)
    
    model = LogisticRegression(C=C, random_state=42, max_iter=1000)
    
    # 5-Fold CV
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    
    return scores.mean()

# 최적화 실행
optuna.logging.set_verbosity(optuna.logging.WARNING)

study = optuna.create_study(direction='maximize')
study.optimize(objective_lr, n_trials=20, show_progress_bar=True)

print(f"\n최적 C 값: {study.best_params['C']:.4f}")
print(f"최적 CV 정확도: {study.best_value:.4f}")

# 최적 모델로 테스트
best_lr = LogisticRegression(C=study.best_params['C'], random_state=42, max_iter=1000)
best_lr.fit(X_train, y_train)
test_accuracy = accuracy_score(y_test, best_lr.predict(X_test))
print(f"테스트 정확도: {test_accuracy:.4f}")

In [ ]:
# 테스트
assert 0.01 <= study.best_params['C'] <= 100, "C 범위 확인"
assert study.best_value >= 0.8, "CV 정확도 80% 이상"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. objective 함수 정의 (trial 객체 받음)
2. trial.suggest_xxx로 파라미터 탐색 공간 정의
3. create_study + optimize로 최적화 실행

**핵심 개념**:
- `suggest_float(name, low, high, log=True)`: 로그 스케일 탐색
- direction='maximize': 최대화 목표 (minimize도 가능)
- study.best_params: 최적 파라미터 딕셔너리

**대안**:
```python
# 정수 파라미터
n_estimators = trial.suggest_int('n_estimators', 10, 200)

# 카테고리 파라미터
solver = trial.suggest_categorical('solver', ['lbfgs', 'saga'])
```

**실무 팁**: n_trials 대신 timeout으로 시간 제한 가능

---

## Q7. Feature Importance 비교 ⭐⭐⭐⭐

**문제**: Random Forest와 Gradient Boosting의 Feature Importance 비교

In [ ]:
# 정답 코드
# 더 복잡한 데이터셋 사용
from sklearn.datasets import make_classification

X_imp, y_imp = make_classification(
    n_samples=500, n_features=8, n_informative=5,
    n_redundant=2, random_state=42
)
feature_names = [f'Feature_{i}' for i in range(8)]

X_train_imp, X_test_imp, y_train_imp, y_test_imp = train_test_split(
    X_imp, y_imp, test_size=0.2, random_state=42
)

# 모델 학습
rf = RandomForestClassifier(n_estimators=100, random_state=42)
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)

rf.fit(X_train_imp, y_train_imp)
gb.fit(X_train_imp, y_train_imp)

# Feature Importance 추출
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'RF_Importance': rf.feature_importances_,
    'GB_Importance': gb.feature_importances_
})

# 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=['Random Forest', 'Gradient Boosting'])

# RF
rf_sorted = importance_df.sort_values('RF_Importance', ascending=True)
fig.add_trace(
    go.Bar(x=rf_sorted['RF_Importance'], y=rf_sorted['Feature'], 
           orientation='h', marker_color='blue', name='RF'),
    row=1, col=1
)

# GB
gb_sorted = importance_df.sort_values('GB_Importance', ascending=True)
fig.add_trace(
    go.Bar(x=gb_sorted['GB_Importance'], y=gb_sorted['Feature'], 
           orientation='h', marker_color='green', name='GB'),
    row=1, col=2
)

fig.update_layout(title='Feature Importance 비교', height=400, showlegend=False)
fig.show()

print("\nFeature Importance 순위 비교:")
print(importance_df.sort_values('RF_Importance', ascending=False))

In [ ]:
# 테스트
assert 'RF_Importance' in importance_df.columns
assert 'GB_Importance' in importance_df.columns
assert importance_df['RF_Importance'].sum() - 1.0 < 0.01, "RF 중요도 합=1"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. 두 모델 학습 후 feature_importances_ 추출
2. DataFrame으로 정리
3. Plotly subplot으로 나란히 시각화

**핵심 개념**:
- feature_importances_: 트리 기반 모델 내장 속성
- RF: 불순도 감소 기반
- GB: 손실 감소 기반

**실무 팁**: 두 모델의 순위가 다를 수 있음 - Permutation Importance로 검증 권장

---

## Q8. SHAP Force Plot 해석 ⭐⭐⭐⭐

**문제**: 테스트 데이터 첫 번째 샘플에 대한 SHAP 값 분석

In [ ]:
# 정답 코드
import shap

# SHAP 계산
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_test_imp[:10])  # 처음 10개만

# 첫 번째 샘플 분석
sample_idx = 0
sample_shap = shap_values[1][sample_idx]  # 클래스 1에 대한 SHAP

# 예측 확률
pred_proba = rf.predict_proba(X_test_imp[sample_idx:sample_idx+1])[0]
actual = y_test_imp[sample_idx]

print(f"샘플 {sample_idx} 분석")
print(f"="*50)
print(f"실제 레이블: {actual}")
print(f"예측 확률: 클래스0={pred_proba[0]:.2%}, 클래스1={pred_proba[1]:.2%}")

# 상위 3개 영향 특성
shap_df = pd.DataFrame({
    'Feature': feature_names,
    'SHAP_Value': sample_shap,
    'Feature_Value': X_test_imp[sample_idx]
}).sort_values('SHAP_Value', key=abs, ascending=False)

print(f"\n상위 3개 영향 특성:")
for i, row in shap_df.head(3).iterrows():
    direction = "이탈 방향(+)" if row['SHAP_Value'] > 0 else "유지 방향(-)"
    print(f"  {row['Feature']}: SHAP={row['SHAP_Value']:.4f} ({direction})")

# 해석 문장
top_feat = shap_df.iloc[0]
print(f"\n해석: '{top_feat['Feature']}'가 예측에 가장 큰 영향을 미쳤으며, ")
print(f"       이 특성의 값이 높아서 {'이탈' if top_feat['SHAP_Value'] > 0 else '유지'} 확률을 높였습니다.")

In [ ]:
# 테스트
assert len(sample_shap) == 8, "8개 특성"
assert shap_df.shape[0] == 8, "모든 특성 포함"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. TreeExplainer로 SHAP 계산
2. 특정 샘플의 SHAP 값 추출
3. 절대값 기준 정렬로 상위 특성 식별
4. 부호로 영향 방향 해석

**핵심 개념**:
- SHAP > 0: 클래스 1(이탈) 방향 기여
- SHAP < 0: 클래스 0(유지) 방향 기여
- 절대값 크기: 영향력 정도

**실무 팁**: SHAP 값의 합 + 기대값 = 예측 확률 (로그오즈)

---

## Q9. PDP 그리기 ⭐⭐⭐⭐⭐

**문제**: 가장 중요한 특성의 PDP + rug plot

In [ ]:
# 정답 코드
from sklearn.inspection import partial_dependence

# 가장 중요한 특성 찾기
top_feature_idx = rf.feature_importances_.argmax()
top_feature_name = feature_names[top_feature_idx]

print(f"가장 중요한 특성: {top_feature_name}")

# PDP 계산
pd_result = partial_dependence(
    rf, X_train_imp, features=[top_feature_idx],
    kind='average', grid_resolution=50
)

# 시각화
fig = go.Figure()

# PDP 라인
fig.add_trace(go.Scatter(
    x=pd_result['grid_values'][0],
    y=pd_result['average'][0],
    mode='lines',
    name='PDP',
    line=dict(width=3, color='blue')
))

# Rug plot (데이터 분포)
fig.add_trace(go.Scatter(
    x=X_train_imp[:, top_feature_idx],
    y=[pd_result['average'][0].min()] * len(X_train_imp),
    mode='markers',
    name='Data Distribution',
    marker=dict(symbol='line-ns', size=10, color='gray', opacity=0.3)
))

fig.update_layout(
    title=f'Partial Dependence Plot: {top_feature_name}',
    xaxis_title=top_feature_name,
    yaxis_title='Predicted Probability',
    height=450
)

fig.show()

In [ ]:
# 테스트
assert pd_result['average'][0].shape[0] == 50, "grid_resolution=50"
assert top_feature_name in feature_names, "유효한 특성명"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. Feature Importance로 가장 중요한 특성 식별
2. partial_dependence로 PDP 계산
3. Plotly로 라인 + rug plot 시각화

**핵심 개념**:
- PDP: 다른 특성 고정, 한 특성 변화 시 예측 변화
- Rug plot: 실제 데이터 분포 표시
- 데이터가 없는 영역의 PDP는 신뢰도 낮음

**실무 팁**: ICE plot(Individual Conditional Expectation)으로 개별 샘플 확인 가능

---

## Q10. 종합: 앙상블 + AutoML + 해석 ⭐⭐⭐⭐⭐

**문제**: 전체 파이프라인 구축

In [ ]:
# 정답 코드
import optuna
import shap
from sklearn.ensemble import StackingClassifier

# 데이터 준비
X_full, y_full = make_classification(
    n_samples=800, n_features=10, n_informative=6,
    n_redundant=2, random_state=42
)
feat_names = [f'Feature_{i}' for i in range(10)]

X_tr, X_te, y_tr, y_te = train_test_split(
    X_full, y_full, test_size=0.2, random_state=42, stratify=y_full
)

print("Step 1: Optuna로 RandomForest 최적화")
print("="*50)

# Step 1: Optuna 최적화
def objective_rf_full(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10)
    }
    model = RandomForestClassifier(**params, random_state=42)
    scores = cross_val_score(model, X_tr, y_tr, cv=3, scoring='roc_auc')
    return scores.mean()

optuna.logging.set_verbosity(optuna.logging.WARNING)
study_full = optuna.create_study(direction='maximize')
study_full.optimize(objective_rf_full, n_trials=10, show_progress_bar=True)

print(f"최적 파라미터: {study_full.best_params}")
print(f"최적 CV AUC: {study_full.best_value:.4f}")

In [ ]:
# Step 2: Stacking 구성
print("\nStep 2: 최적 RF + LR + GB로 Stacking")
print("="*50)

best_rf_model = RandomForestClassifier(**study_full.best_params, random_state=42)

stacking_final = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42, max_iter=1000)),
        ('rf_opt', best_rf_model),
        ('gb', GradientBoostingClassifier(n_estimators=50, random_state=42))
    ],
    final_estimator=LogisticRegression(random_state=42),
    cv=5
)

stacking_final.fit(X_tr, y_tr)
y_pred_final = stacking_final.predict(X_te)
y_proba_final = stacking_final.predict_proba(X_te)[:, 1]

auc_final = roc_auc_score(y_te, y_proba_final)
acc_final = accuracy_score(y_te, y_pred_final)

print(f"Stacking AUC: {auc_final:.4f}")
print(f"Stacking Accuracy: {acc_final:.4f}")

In [ ]:
# Step 3: SHAP 분석
print("\nStep 3: SHAP로 상위 3개 중요 특성")
print("="*50)

# 최적 RF 모델로 SHAP 계산
best_rf_model.fit(X_tr, y_tr)
explainer_final = shap.TreeExplainer(best_rf_model)
shap_vals = explainer_final.shap_values(X_te[:50])

# 평균 절대 SHAP
mean_shap = np.abs(shap_vals[1]).mean(axis=0)
shap_importance = pd.DataFrame({
    'Feature': feat_names,
    'Mean_SHAP': mean_shap
}).sort_values('Mean_SHAP', ascending=False)

print("상위 3개 중요 특성:")
for i, row in shap_importance.head(3).iterrows():
    print(f"  {row['Feature']}: {row['Mean_SHAP']:.4f}")

In [ ]:
# 최종 요약
print("\n" + "="*50)
print("최종 요약")
print("="*50)
print(f"최적 RF 파라미터: {study_full.best_params}")
print(f"Stacking AUC: {auc_final:.4f}")
print(f"상위 중요 특성: {', '.join(shap_importance.head(3)['Feature'].tolist())}")

In [ ]:
# 테스트
assert study_full.best_value >= 0.7, "Optuna 최적화 성공"
assert auc_final >= 0.8, "Stacking AUC 80% 이상"
assert shap_importance.shape[0] == 10, "모든 특성 SHAP 계산"
print("모든 테스트 통과!")

### 풀이 설명

**접근 방법**:
1. Optuna로 RF 하이퍼파라미터 최적화 (10 trials)
2. 최적 RF + LR + GB로 Stacking 앙상블 구성
3. TreeExplainer로 SHAP 계산
4. 평균 절대 SHAP로 상위 특성 식별

**핵심 개념**:
- Optuna: 효율적인 하이퍼파라미터 탐색
- Stacking: 최고 성능 앙상블
- SHAP: 모델 불가지론적 해석

**실무 팁**:
1. Optuna trials 수는 시간과 성능 트레이드오프
2. Stacking의 메타 모델은 간단한 것이 좋음
3. SHAP은 계산 비용이 크므로 샘플링 사용

---

## 학습 정리

### 퀴즈별 핵심 개념

| 퀴즈 | 난이도 | 핵심 개념 |
|-----|-------|----------|
| Q1 | ⭐ | Hard Voting = 다수결 |
| Q2 | ⭐⭐ | Soft Voting = 확률 평균 |
| Q3 | ⭐⭐ | VotingClassifier 구성 |
| Q4 | ⭐⭐⭐ | Stacking vs Voting 비교 |
| Q5 | ⭐⭐⭐ | 가중 Voting 최적화 |
| Q6 | ⭐⭐⭐ | Optuna 기본 사용 |
| Q7 | ⭐⭐⭐⭐ | Feature Importance 비교 |
| Q8 | ⭐⭐⭐⭐ | SHAP 개별 예측 해석 |
| Q9 | ⭐⭐⭐⭐⭐ | PDP + rug plot |
| Q10 | ⭐⭐⭐⭐⭐ | 종합 파이프라인 |

### Kaggle 대회 적용 팁

1. **베이스라인**: 단일 모델로 시작
2. **앙상블 순서**: Voting -> Stacking -> Blending
3. **Optuna 활용**: timeout 설정으로 시간 관리
4. **SHAP 필수**: 발표 시 모델 설명에 활용
5. **다양성 확보**: 다른 알고리즘, 다른 전처리 조합